<a href="https://colab.research.google.com/github/rubengp39/AirQuality-Raspi/blob/main/IOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
import json
import requests
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [8]:
# Opening JSON file
jsonArr = []
with open('weatherData.json') as json_file:
    data = json.load(json_file)

In [9]:
columns = ["last_updated", "temp_c", "wind_kph", "wind_degree", "pressure_mb", "precip_mm", "humidity", "cloud", "pm2_5", "pm10"]
df = pd.DataFrame(columns = columns)
for i in range(0, len(data)):
  row = []
  row.append(data[i]["current"]['last_updated'])
  row.append(data[i]["current"]['temp_c'])
  row.append(data[i]["current"]['wind_kph'])
  row.append(data[i]["current"]['wind_degree'])
  row.append(data[i]["current"]['pressure_mb'])
  row.append(data[i]["current"]['precip_mm'])
  row.append(data[i]["current"]['humidity'])
  row.append(data[i]["current"]['cloud'])
  row.append(data[i]["current"]['air_quality']["pm2_5"])
  row.append(data[i]["current"]['air_quality']["pm10"])
  df = df.append(dict(zip(df.columns, row)), ignore_index=True)


In [10]:
x_values = df[["temp_c", 	"wind_kph",	"precip_mm",	"humidity"]].values
y2_5 = df["pm2_5"].values
y10 = df["pm10"].values
degree = 3

#Model pm2_5
poly_model2_5 = PolynomialFeatures(degree=degree)
poly_x_values = poly_model2_5.fit_transform(x_values)
poly_model2_5.fit(poly_x_values, y2_5)

regression_model2_5 = LinearRegression()
regression_model2_5.fit(poly_x_values, y2_5)

#Model pm10
poly_model10 = PolynomialFeatures(degree=degree)
poly_x_values = poly_model10.fit_transform(x_values)
poly_model10.fit(poly_x_values, y10)

regression_model10 = LinearRegression()
regression_model10.fit(poly_x_values, y10)

LinearRegression()

In [11]:
URL = "http://api.weatherapi.com/v1/forecast.json?key=3817edbde7fd4412a26182206221312&q=bilbao&aqi=yes"

r = requests.get(url=URL)
data = r.json()
forecastData = data["forecast"]["forecastday"][0]["hour"]

In [12]:
x_pred = []

for i in range(24):
  x_pred.append([forecastData[i]["temp_c"], forecastData[i]["wind_kph"], forecastData[i]["precip_mm"], forecastData[i]["humidity"]])

#pm2_5 pred
poly_x_values_pred = poly_model2_5.fit_transform(x_pred)
y2_5_pred = regression_model2_5.predict(poly_x_values_pred)
for i in range(len(y2_5_pred)):
  if y2_5_pred[i]<0:
    y2_5_pred[i]=0

#pm10 pred
poly_x_values_pred = poly_model10.fit_transform(x_pred)
y10_pred = regression_model10.predict(poly_x_values_pred)
for i in range(len(y10_pred)):
  if y10_pred[i]<0:
    y10_pred[i]=0

#Forecast display
for i in range(24):
  print("At", i, ": 00 tomorrow, the forecast is : \n\n     temperature (°C) :", x_pred[i][0], "\n     wind speed (kph) :", x_pred[i][1], "\n     precipitations (mm) :", x_pred[i][2], "\n     humidity (%) :", x_pred[i][3], "\n\n     air quality : \n          pm2_5 :", y2_5_pred[i], "\n          pm10 :", y10_pred[i],"\n\n")

At 0 : 00 tomorrow, the forecast is : 

     temperature (°C) : 11.4 
     wind speed (kph) : 25.2 
     precipitations (mm) : 0.0 
     humidity (%) : 65 

     air quality : 
          pm2_5 : 0.0 
          pm10 : 0.0 


At 1 : 00 tomorrow, the forecast is : 

     temperature (°C) : 10.9 
     wind speed (kph) : 24.5 
     precipitations (mm) : 0.0 
     humidity (%) : 67 

     air quality : 
          pm2_5 : 0.0 
          pm10 : 0.0 


At 2 : 00 tomorrow, the forecast is : 

     temperature (°C) : 10.5 
     wind speed (kph) : 25.6 
     precipitations (mm) : 0.0 
     humidity (%) : 68 

     air quality : 
          pm2_5 : 0.0 
          pm10 : 0.0 


At 3 : 00 tomorrow, the forecast is : 

     temperature (°C) : 10.0 
     wind speed (kph) : 25.6 
     precipitations (mm) : 0.0 
     humidity (%) : 70 

     air quality : 
          pm2_5 : 1.1328125953300514 
          pm10 : 1.6586017281512113 


At 4 : 00 tomorrow, the forecast is : 

     temperature (°C) : 10.1 
    